In [ ]:
import pandas as pd
import extruct
from usp.tree import sitemap_tree_for_homepage
import nest_asyncio
import asyncio
import aiohttp
from w3lib.html import get_base_url
import pickle

In [ ]:
nest_asyncio.apply()

In [ ]:
tree = sitemap_tree_for_homepage('https://www.feedbacksports.com')

In [ ]:
pages = []

for page in tree.all_pages():
    pages.append(page.url)
    
master_dict = {}

In [7]:
# This is our asynchronous web scraper
def async_code(urls, nested_links):
    
    master_dict = {}
    
    timeout = aiohttp.ClientTimeout(total=30)
    all_data = []

    async def extract_microdata(url, text):
        base_url = get_base_url(text, url)
        data = extruct.extract(text, base_url=base_url)
        return data

    async def fetch(session, url, sem):
        async with sem:
            try:
                async with session.get(url, timeout=timeout) as response:
                    text = await response.text()
                    microdata = await extract_microdata(url, text)
                    return text, url, microdata
            except Exception as e:
                print(str(e))

    async def main(urls, nested_links):
        tasks = []
        sem = asyncio.Semaphore(15)
        headers = {
            "user-agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
        async with aiohttp.ClientSession(headers=headers) as session:
            if nested_links==True:
                for serp_data in urls:
                    for url in serp_data:
                        tasks.append(fetch(session, url, sem))
            else:
                for url in urls:
                    tasks.append(fetch(session, url, sem))

            htmls = await asyncio.gather(*tasks)
            all_data.extend(htmls)

            # Storing the raw HTML data.
            for html in htmls:
                if html is not None:
                    url = html[1]
                    master_dict[url] = {'Raw Html': html[0]}
                    master_dict[url]['Microdata'] = html[2]
                else:
                    continue

            # Accomodating for URL's that fail.
            complete_urls = list(master_dict.keys())
            if nested_links == True:
                for serp_data in urls:
                    for url in serp_data:
                        if url not in complete_urls:
                            master_dict[url] = np.nan
                        else:
                            pass
            else:
                for url in urls:
                    if url not in complete_urls:
                        master_dict[url] = np.nan
                    else:
                        pass

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main(urls, nested_links))
    
    return master_dict

# 2. Get The Extruct Data For All Of The URLS:

In [8]:
data = async_code(pages, nested_links=False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The object should be created from async function


'charmap' codec can't decode byte 0x9d in position 82109: character maps to <undefined>
'charmap' codec can't decode byte 0x9d in position 75831: character maps to <undefined>








[Errno 32] Broken pipe
[Errno 32] Broken pipe























































































'charmap' codec can't decode byte 0x9d in position 142015: character maps to <undefined>

'charmap' codec can't decode byte 0x9d in position 82286: character maps to <undefined>


















































































































'charmap' codec can't decode byte 0x9d in position 82380: character maps to <undefined>
'charmap' codec can't decode byte 0x9d in position 142015: character maps to <undefined>








NameError: name 'np' is not defined

In [ ]:
with open(filename, 'wb') as f:
    pickle.dump(data, f)

# 3. Save The Data To A DataFrame And Combine The Schema Data With The URL Data:

In [ ]:
df = pd.DataFrame(pages)
df.rename(columns={0:'URLs'}, inplace=True)

------------------------------------

------------------------------------------------------------------------------------------------------------